### <font color='gold'>「从这里开始」</font>数据集构建

In [8]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import os
import torch
pdbbind_path = "/home/jovyan/dataspace/PDBbind2020/all_pdbbind"
proto_path = "/home/jovyan/dataspace/Prototype"
p2rank = "bash /home/jovyan/TankBind/p2rank_2.3/prank"
tankbind_src_folder_path = "../tankbind_prototype/"
import sys
sys.path.insert(0, tankbind_src_folder_path)

In [12]:
info = pd.read_pickle("/home/jovyan/dataspace/Prototype/tables/SMALL_info_core_pocket.pkl")

In [12]:
protein_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/SMALL_protein_dict.pt")
compound_dict = torch.load("/home/jovyan/dataspace/Prototype/dicts/SMALL_compound_torchdrug_features.pt")

In [13]:
protein_dict.keys()

dict_keys(['1i5r', '3hb4', '1w6j', '3rde', '5lp1'])

In [14]:
from data_prototype import TankBindDataSet_prototype

toFileSmall = f"/home/jovyan/main_tankbind/dataset_prototype/small"
os.system(f"rm -rf {toFileSmall}")
os.system(f"mkdir -p {toFileSmall}")
dataset = TankBindDataSet_prototype(toFileSmall, data=info, protein_dict=protein_dict, compound_dict=compound_dict)

Processing...


['/home/jovyan/main_tankbind/dataset_prototype/small/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small/processed/compound.pt']


Done!


In [15]:
t = []
t_dict = {}
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    t_dict[sample_id] = [i, sample_id, p_length, c_length, y_length]
    
torch.save(t, "/home/jovyan/dataspace/Prototype/tables/SMALL_supplementary.pt")
torch.save(t_dict, "/home/jovyan/dataspace/dicts/tables/SMALL_supplementary_dict.pt")

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])
t.to_csv("/home/jovyan/dataspace/Prototype/tables/SMALL_supplementary.csv")

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFileSmall}/processed/data.pt")

  0%|          | 0/55 [00:00<?, ?it/s]

In [16]:
toFile = "/home/jovyan/main_tankbind/dataset_prototype/small_train"
os.system(f"rm -rf {toFile}")
os.system(f"mkdir -p {toFile}")
dataset = TankBindDataSet_prototype(toFile, data=info[info.split_tag=="train"].reset_index(drop=True), protein_dict=protein_dict, compound_dict=compound_dict)

t = []
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFile}/processed/data.pt")

Processing...


['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']


Done!


  0%|          | 0/51 [00:00<?, ?it/s]

In [17]:
toFile = "/home/jovyan/main_tankbind/dataset_prototype/small_val"
os.system(f"rm -rf {toFile}")
os.system(f"mkdir -p {toFile}")
dataset = TankBindDataSet_prototype(toFile, data=info[info.split_tag!="train"].reset_index(drop=True), protein_dict=protein_dict, compound_dict=compound_dict)

t = []
data = dataset.data

for i, line in tqdm(data.iterrows(), total=data.shape[0]):
    d = dataset[i]
    sample_id = line['sample_id']
    p_length = d['node_xyz'].shape[0]
    c_length = d['coords'].shape[0]
    y_length = d['y'].shape[0]
    t.append([i, sample_id, p_length, c_length, y_length])
    

t = pd.DataFrame(t, columns=['index', 'sample_id', 'p_length', 'c_length', 'y_length'])

data = pd.concat([data, t[['p_length', 'c_length', 'y_length']]], axis=1)
torch.save(data, f"{toFile}/processed/data.pt")

Processing...
Done!


['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']


  0%|          | 0/4 [00:00<?, ?it/s]

In [18]:
def get_data_prototype(pre, data_mode, addNoise=None):
    if data_mode == "0":
        add_noise_to_com = None
        
        train_dataset = TankBindDataSet_prototype(root=f"{pre}/small_train", add_noise_to_com=None)
        train_dataset.data = train_dataset.data.query("c_length < 100").reset_index(drop=True)
        train_dataset.data = train_dataset.data.groupby("session_aus").sample(n=1).reset_index(drop=True)
        
        iid_dataset = TankBindDataSet_prototype(root=f"{pre}/small_val", add_noise_to_com=None)
        iid_index = iid_dataset.data.query("split_tag =='iid_val'").index.values
        iid_dataset.data = iid_dataset.data.iloc[iid_index].reset_index(drop=True) if len(iid_index>0) else None
        
        ood_dataset = TankBindDataSet_prototype(root=f"{pre}/small_val", add_noise_to_com=None)
        ood_index = ood_dataset.data.query("split_tag =='ood_val'").index.values
        ood_dataset.data = ood_dataset.data.iloc[ood_index].reset_index(drop=True) if len(ood_index>0) else None
        
        test_dataset = TankBindDataSet_prototype(root=f"{pre}/small_val", add_noise_to_com=None)
        test_index = test_dataset.data.query("split_tag =='test'").index.values
        test_dataset.data = test_dataset.data.iloc[test_index].reset_index(drop=True) if len(test_index>0) else None       

    return train_dataset, iid_dataset, ood_dataset, test_dataset

In [19]:
input_path = "/home/jovyan/main_tankbind/dataset_prototype"
train_dataset, iid_dataset, ood_dataset, test_dataset = get_data_prototype(input_path, "0")
print(len(train_dataset.data))

['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']
45


In [20]:
pre = "/home/jovyan/main_tankbind/dataset_prototype"
train_dataset, iid_dataset, ood_dataset, test_dataset = get_data_prototype(pre, "0")

['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']


In [21]:
from sampler_prototype import SessionBatchSampler
from torch_geometric.loader import DataLoader



In [22]:
a = np.arange(14)

In [23]:
a[-3:]

array([11, 12, 13])

In [24]:
train_batch_sampler = SessionBatchSampler(train_dataset, seed=0, index_save_path="/home/jovyan/main_tankbind/training_information_prototype/small_train")
train_loader = DataLoader(train_dataset,
                        follow_batch=['x', 'compound_pair'], 
                        batch_sampler=train_batch_sampler, 
                        pin_memory=False, 
                        num_workers=8)

SessionBatchSampler | Refreshing batches with seed 0 for epoch 0.
SessionBatchSampler | Saving all samples' group indices.
[[0, 1], [12, 3, 10, 20, 16, 18], [8, 6, 4, 7, 15, 11], [9, 19, 13, 5, 2, 17], [21, 22, 23, 24, 25], [44, 42, 36, 28, 43, 39], [37, 26, 29, 30, 38, 41], [35, 31, 27, 34, 40, 33]]
SessionBatchSampler | Saving group indices for epoch 0.


In [25]:
train_loader
train_loader.batch_sampler.prepare_batches_for_epoch(epoch=0)
for data in tqdm(train_loader):
    print("Session:",data.session_au)
    print("Sample_id:", data.sample_id)
    print(data.value,"\n")
train_loader.batch_sampler.prepare_batches_for_epoch(epoch=2)
for data in tqdm(train_loader):
    print("Session:",data.session_au)
    print("Sample_id:", data.sample_id)
    print(data.value,"\n")

SessionBatchSampler | Refreshing batches with seed 0 for epoch 0.
[[0, 1], [12, 3, 10, 20, 16, 18], [8, 6, 4, 7, 15, 11], [9, 19, 13, 5, 2, 17], [21, 22, 23, 24, 25], [44, 42, 36, 28, 43, 39], [37, 26, 29, 30, 38, 41], [35, 31, 27, 34, 40, 33]]
SessionBatchSampler | Saving group indices for epoch 0.


  0%|          | 0/8 [00:00<?, ?it/s]

Session: ['101010_Q13093', '101010_Q13093']
Sample_id: ['ID_40461', 'ID_40462']
tensor([ 1., 15.], dtype=torch.float64) 

Session: ['208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469']
Sample_id: ['ID_44', 'ID_33', 'ID_42', 'ID_53', 'ID_49', 'ID_51']
tensor([10000.,  2000.,    50.,  5000., 10000., 10000.], dtype=torch.float64) 

Session: ['208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469']
Sample_id: ['ID_40', 'ID_38', 'ID_36', 'ID_39', 'ID_48', 'ID_43']
tensor([ 2500., 10000.,   100.,    40.,   320.,    35.], dtype=torch.float64) 

Session: ['208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469']
Sample_id: ['ID_41', 'ID_52', 'ID_45', 'ID_37', 'ID_32', 'ID_50']
tensor([   60.,  5000.,   320., 10000., 10000., 10000.], dtype=torch.float64) 

Session: ['52_P14061', '52_P14061', '52_P14061', '52_P14061', '52_P14061']
Sample_id: ['ID_1', 'ID_2', 'ID_5', 'ID_7', 'ID_9']
tensor([ 460., 1000., 4300.,  4

  0%|          | 0/8 [00:00<?, ?it/s]

Session: ['101010_Q13093', '101010_Q13093']
Sample_id: ['ID_40461', 'ID_40462']
tensor([ 1., 15.], dtype=torch.float64) 

Session: ['208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469']
Sample_id: ['ID_43', 'ID_38', 'ID_49', 'ID_32', 'ID_44', 'ID_39']
tensor([   35., 10000., 10000., 10000., 10000.,    40.], dtype=torch.float64) 

Session: ['208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469']
Sample_id: ['ID_37', 'ID_33', 'ID_47', 'ID_41', 'ID_52', 'ID_51']
tensor([10000.,  2000.,   450.,    60.,  5000., 10000.], dtype=torch.float64) 

Session: ['208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469', '208_P16469']
Sample_id: ['ID_36', 'ID_45', 'ID_40', 'ID_53', 'ID_48', 'ID_50']
tensor([  100.,   320.,  2500.,  5000.,   320., 10000.], dtype=torch.float64) 

Session: ['52_P14061', '52_P14061', '52_P14061', '52_P14061', '52_P14061']
Sample_id: ['ID_1', 'ID_2', 'ID_5', 'ID_7', 'ID_9']
tensor([ 460., 1000., 4300.,  4

In [21]:
data.value

tensor([11.4000, 15.7000,  4.1000,  5.6000, 22.5000,  7.9000,  7.8000,  6.3000],
       dtype=torch.float64)

In [23]:
data.session_au

['80_P48449',
 '80_P48449',
 '80_P48449',
 '80_P48449',
 '80_P48449',
 '80_P48449',
 '80_P48449',
 '80_P48449']

In [1]:
import numpy as np

In [20]:
for grp in info:
    print("GROUP")
    for _ in grp: print(train_dataset[_].session_au, train_dataset[_].sample_id)

GROUP
101010_Q13093 ID_40461
101010_Q13093 ID_40462
GROUP
208_P16469 ID_41
208_P16469 ID_53
208_P16469 ID_37
208_P16469 ID_45
208_P16469 ID_44
208_P16469 ID_33
208_P16469 ID_43
208_P16469 ID_32
GROUP
52_P14061 ID_0
52_P14061 ID_2
52_P14061 ID_5
52_P14061 ID_7
52_P14061 ID_9
GROUP
80_P48449 ID_17
80_P48449 ID_27
80_P48449 ID_11
80_P48449 ID_29
80_P48449 ID_31
80_P48449 ID_28
80_P48449 ID_23
80_P48449 ID_15


In [304]:
info = train_dataset.data.groupby("session_au").sample(n=8, replace=True, random_state=seed).drop_duplicates(subset="sample_id")
batch_indices = info.groupby("session_au").indices

In [305]:
batch_indices

{'101010_Q13093': array([0, 1]),
 '208_P16469': array([2, 3, 4, 5, 6, 7]),
 '52_P14061': array([ 8,  9, 10, 11, 12]),
 '80_P48449': array([13, 14, 15, 16, 17, 18])}

In [309]:
for key in batch_indices:
    print("KEY=", key)
    for _ in batch_indices[key]:
        print(_, train_dataset[_].session_au)

    

KEY= 101010_Q13093
0 101010_Q13093
1 101010_Q13093
KEY= 208_P16469
2 208_P16469
3 208_P16469
4 208_P16469
5 208_P16469
6 208_P16469
7 208_P16469
KEY= 52_P14061
8 208_P16469
9 208_P16469
10 208_P16469
11 208_P16469
12 208_P16469
KEY= 80_P48449
13 208_P16469
14 208_P16469
15 208_P16469
16 208_P16469
17 208_P16469
18 208_P16469


In [301]:
batch_indices
for key in batch_indices.keys():
    idx = list(batch_indices[key])
    yield idx

{'101010_Q13093': array([0, 1]),
 '208_P16469': array([2, 3, 4, 5, 6, 7]),
 '52_P14061': array([ 8,  9, 10, 11, 12]),
 '80_P48449': array([13, 14, 15, 16, 17, 18])}

In [270]:
train_2 = train_dataset[dd['208_P16469']]
for _ in train_2: print(_.session_au)

208_P16469
208_P16469
208_P16469
208_P16469
208_P16469
208_P16469
208_P16469
208_P16469


In [ ]:
for i in dd['208_P16469']:
    print(train_)

In [258]:
for i in range(len(train_dataset)):
    print(i, train_dataset[i].sample_id, train_dataset.data.iloc[i].sample_id)

0 ID_32 ID_32
1 ID_33 ID_33
2 ID_36 ID_36
3 ID_37 ID_37
4 ID_38 ID_38
5 ID_39 ID_39
6 ID_40 ID_40
7 ID_41 ID_41
8 ID_42 ID_42
9 ID_43 ID_43
10 ID_44 ID_44
11 ID_45 ID_45
12 ID_46 ID_46
13 ID_48 ID_48
14 ID_49 ID_49
15 ID_50 ID_50
16 ID_51 ID_51
17 ID_52 ID_52
18 ID_53 ID_53
19 ID_1 ID_1
20 ID_3 ID_3
21 ID_5 ID_5
22 ID_7 ID_7
23 ID_9 ID_9
24 ID_11 ID_11
25 ID_12 ID_12
26 ID_14 ID_14
27 ID_15 ID_15
28 ID_16 ID_16
29 ID_17 ID_17
30 ID_18 ID_18
31 ID_19 ID_19
32 ID_20 ID_20
33 ID_21 ID_21
34 ID_22 ID_22
35 ID_23 ID_23
36 ID_24 ID_24
37 ID_26 ID_26
38 ID_27 ID_27
39 ID_28 ID_28
40 ID_29 ID_29
41 ID_30 ID_30
42 ID_31 ID_31


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import hashlib

from tqdm import tqdm
import rdkit.Chem as Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import glob
import torch
from torch_geometric.loader import DataLoader
from metrics import *
from utils import *
from datetime import datetime
import logging
import sys
import argparse
from torch.utils.data import RandomSampler
from torch.utils.data import WeightedRandomSampler
import random
import math
from torch.utils.tensorboard import SummaryWriter
from data_prototype import get_data_prototype
from sampler_prototype import SessionBatchSampler
from model import *

class PairwiseLoss(nn.Module):
    def __init__(self, keep_rate=1., sigmoid_lambda=1.,
                 ingrp_thr=0.3, outgrp_thr=9999, eval=False):
        super(PairwiseLoss, self).__init__()
        self.eval = eval
        self.register_buffer('keep_rate', torch.tensor(keep_rate, dtype=torch.float64))
        self.register_buffer('sigmoid_lambda', torch.tensor(sigmoid_lambda, dtype=torch.float64))
        self.register_buffer('ingrp_thr', torch.tensor(ingrp_thr, dtype=torch.float64))
        self.register_buffer('outgrp_thr', torch.tensor(outgrp_thr, dtype=torch.float64))

    def forward(self, pred, true,): # groupid):

        """
        Customized pairwise ranking loss.

        """
        if len(pred.shape) == 1:
            pred = pred.unsqueeze(1)
        if len(true.shape) == 1:
            true = true.unsqueeze(1)
        drop_rate = 1 - self.keep_rate
        true_tile_row = true.repeat((1, true.shape[0]))
        true_tile_col = torch.t(true_tile_row)
        assert (true_tile_row.shape == true_tile_col.shape)

        pred_tile_row = pred.repeat((1, pred.shape[0]))
        pred_tile_col = torch.t(pred_tile_row)
    
        diff = (true_tile_row - true_tile_col) / (torch.abs(true_tile_col) + 1e-4)

        pred_pair = torch.stack([pred_tile_row, pred_tile_col], dim=0)
        valid_ind = torch.logical_or(diff > self.ingrp_thr,
                                     diff > self.outgrp_thr)
        
        # print("len(valid_ind)",len(valid_ind))
        
        pred_pair_valid = pred_pair.masked_select(valid_ind).reshape(2, -1)
        pred_pair_diff = pred_pair_valid[0] - pred_pair_valid[1]
        reverse = torch.sum(pred_pair_diff > 0)
        ntotal = pred_pair_valid.shape[1] + 1e-8
        #print("Total valid pairs: {:.3f}, reversed: {:.3f}, reverse ratio: {:.3f}".format(ntotal, reverse, reverse/ntotal))
        if drop_rate > 1e-4:
            pred_pair_valid_dropout = torch.nn.functional.dropout(pred_pair_valid[0], drop_rate) * self.keep_rate
            pred_pair_valid_ind = torch.logical_or(pred_pair_valid_dropout == pred_pair_valid[0],
                                                   pred_pair_valid_dropout != 0.0)

            pred_pair_valid = pred_pair_valid.masked_select(pred_pair_valid_ind).reshape(2, -1)
        # print(pred_pair_valid[1] - pred_pair_valid[0])
        loss = torch.sum(torch.log(1. + torch.exp(self.sigmoid_lambda * (pred_pair_valid[1] - pred_pair_valid[0]))))
        num = pred_pair_valid.shape[1] + 1e-8
        loss = torch.div(loss, num)
        return loss, (reverse/ntotal)

In [239]:
train_dataset.data

,sample_id,pdb_id,uniprot_id,assay_id,smiles_name,kekule_smiles,session_au,session_aus,pocket_com,pocket_name,value,docking_score,mmgbsa_binding_energy,qed,split_tag,p2rank_core_pocket,session_au_usability,p_length,c_length,y_length
0,ID_0,1I5R,P14061,52,smiles_0,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,460.0,-10.036025,-69.084900,0.709065,train,True,multiton,174,24,4176
1,ID_2,1I5R,P14061,52,smiles_1,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,1000.0,-7.643840,-44.594658,0.766057,train,True,multiton,174,24,4176
2,ID_5,3HB4,P14061,52,smiles_2,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,4300.0,-8.924538,-54.790604,0.850050,train,True,multiton,164,24,3936
3,ID_7,1I5R,P14061,52,smiles_3,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,420.0,-7.099580,-40.027607,0.510305,train,True,multiton,174,24,4176
4,ID_9,3HB4,P14061,52,smiles_4,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,1150.0,-9.107697,-61.683292,0.587624,train,True,multiton,164,26,4264
5,ID_11,1W6J,P48449,80,smiles_6,Br.C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=CC=C2)C=C1,80_P48449,80_P48449_Br.C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC...,"[[29.8353, 70.7209, 9.1415]]",1w6j_0,22.5,-9.244177,-60.514454,0.250173,train,True,multiton,205,27,5535
6,ID_12,1W6J,P48449,80,smiles_7,C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(Br)C=C2)C(...,80_P48449,80_P48449_C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(...,"[[29.8353, 70.7209, 9.1415]]",1w6j_0,6.5,-11.721857,-68.603218,0.133516,train,True,multiton,205,36,7380
7,ID_14,1W6J,P48449,80,smiles_9,C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(Br)C=C2)C(...,80_P48449,80_P48449_C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(...,"[[29.8353, 70.7209, 9.1415]]",1w6j_0,6.3,-11.261114,-65.704399,0.124983,train,True,multiton,205,36,7380
8,ID_15,1W6J,P48449,80,smiles_10,C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(Br)C=C2)C(...,80_P48449,80_P48449_C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(...,"[[29.8353, 70.7209, 9.1415]]",1w6j_0,4.6,-10.448766,-74.914139,0.124495,train,True,multiton,205,37,7585
9,ID_16,1W6J,P48449,80,smiles_11,C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(Br)C=C2)C(...,80_P48449,80_P48449_C=CCN(C)CCCCCCOC1=CC=C(C(=O)C2=CC=C(...,"[[29.8353, 70.7209, 9.1415]]",1w6j_0,6.2,-11.113625,-83.528900,0.083234,train,True,multiton,205,37,7585


In [241]:
for i in range(len(train_dataset)):
    print(i, train_dataset[i].sample_id)

0 ID_0
1 ID_5
2 ID_11
3 ID_14
4 ID_16
5 ID_17
6 ID_18
7 ID_19
8 ID_20
9 ID_21
10 ID_22
11 ID_23
12 ID_24
13 ID_26
14 ID_27
15 ID_28
16 ID_29
17 ID_30
18 ID_31
19 ID_32
20 ID_33
21 ID_36
22 ID_37
23 ID_38
24 ID_39
25 ID_40
26 ID_41
27 ID_42
28 ID_43
29 ID_44
30 ID_45
31 ID_47
32 ID_48
33 ID_49
34 ID_50
35 ID_51
36 ID_53


IndexError: single positional indexer is out-of-bounds

In [215]:
p = train_dataset.data.reset_index(drop=True)

In [167]:
train_dataset = TankBindDataSet_prototype(root=f"{pre}/small_train", add_noise_to_com=None)
train_dataset.data = train_dataset.data.query("c_length < 100").reset_index(drop=True)


['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']


In [168]:
train_dataset[5]

HeteroData(
  dis_map=[3936],
  node_xyz=[164, 3],
  coords=[24, 3],
  y=[3936],
  seq=[164],
  compound_pair=[576, 16],
  pdb_id='3HB4',
  sample_id='ID_5',
  split_tag='train',
  session_au='52_P14061',
  real_affinity_mask=[1],
  real_y_mask=[3936],
  protein={
    node_s=[164, 6],
    node_v=[164, 3, 3]
  },
  compound={ x=[24, 56] },
  (protein, p2p, protein)={
    edge_index=[2, 3837],
    edge_s=[3837, 32],
    edge_v=[3837, 1, 3]
  },
  (compound, c2c, compound)={
    edge_index=[2, 54],
    edge_weight=[54],
    edge_attr=[54, 19]
  }
)

In [176]:
d = train_dataset.data.groupby("session_aus").sample(n=1)

In [172]:
e = d.sort_index()

In [178]:
train_index = d.query("split_tag =='train'").sort_index().index.values
train_index

array([ 1,  2,  4,  6,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 42, 43, 44, 45, 46, 47, 48])

In [179]:
train_2 = train_dataset[train_index]

In [186]:
train_2[0].sample_id, train_2[1].sample_id, train_2[2].sample_id, train_2[3].sample_id, train_2[4].sample_id, train_2[5].sample_id, train_2[6].sample_id, 

('ID_1', 'ID_2', 'ID_4', 'ID_6', 'ID_8', 'ID_11', 'ID_12')

In [188]:
train_2.data.loc[0].sample_id, train_2.data.loc[1].sample_id, train_2.data.loc[2].sample_id, train_2.data.loc[3].sample_id, train_2.data.loc[4].sample_id, train_2.data.loc[5].sample_id, train_2.data.loc[6].sample_id

('ID_0', 'ID_1', 'ID_2', 'ID_3', 'ID_4', 'ID_5', 'ID_6')

In [ ]:
train_index = d.query("split_tag =='train'").index.values
train_index = train_index.sort()
btrain_dataset = train_dataset[train_index]

In [166]:
pre = "/home/jovyan/main_tankbind/dataset_prototype"
train_dataset, iid_dataset, ood_dataset, test_dataset = get_data_prototype(pre, "0")

['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']


IndexError: Only slices (':'), list, tuples, torch.tensor and np.ndarray of dtype long or bool are valid indices (got 'NoneType')

In [164]:
train_dataset[5].sample_id
d = train_dataset.data.sort_index()
train_dataset[5].sample_id, d.loc[5]

('ID_11',
 sample_id                                                             ID_5
 pdb_id                                                                3HB4
 uniprot_id                                                          P14061
 assay_id                                                                52
 smiles_name                                                       smiles_2
 kekule_smiles            C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...
 session_au                                                       52_P14061
 session_aus              52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...
 pocket_com                                     [[13.393, 3.0706, -6.9976]]
 pocket_name                                                         3hb4_0
 value                                                               4300.0
 docking_score                                                    -8.924538
 mmgbsa_binding_energy                                           -54.790604
 q

In [141]:
train_info.reset_index(inplace=True)
train_info.loc[5]

index                                                                   34
sample_id                                                            ID_39
pdb_id                                                                3RDE
uniprot_id                                                          P16469
assay_id                                                               208
smiles_name                                                      smiles_34
kekule_smiles                      CCCCCCCCCC/C=C/COCC1=CC=C(CCC(=O)O)C=C1
session_au                                                      208_P16469
session_aus              208_P16469_CCCCCCCCCC/C=C/COCC1=CC=C(CCC(=O)O)...
pocket_com                                [[-11.3368, -41.5066, -45.8404]]
pocket_name                                                         3rde_0
value                                                                 40.0
docking_score                                                    -7.061187
mmgbsa_binding_energy    

In [110]:
if True:
    add_noise_to_com = None
        
    train_dataset = TankBindDataSet_prototype(root=f"{pre}/small_train", add_noise_to_com=None)
    train_dataset.data = train_dataset.data.query("c_length < 100").reset_index(drop=True)


['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']


In [113]:
if True:
    d = train_dataset.data.groupby("session_aus").sample(n=1)
    train_index = d.query("split_tag =='train'").index.values

In [117]:
train_index.sort()
train_index

array([ 1,  2,  5,  7,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 43, 44, 45, 46, 47, 48])

In [125]:
if True:
    d = train_dataset.data.groupby("session_aus").sample(n=1)
    
    train_info = d.query("split_tag =='train'")
    train_index = train_info.index.values
    train_dataset = train_dataset[train_index]
        
        
    val_test_dataset = TankBindDataSet_prototype(root=f"{pre}/small_val", add_noise_to_com=None)
    val_test_dataset.data = val_test_dataset.data.query("c_length < 100").reset_index(drop=True)
    d = val_test_dataset.data
        
    
    
    
        
    iid_info = d.query("split_tag =='iid_val'")
    iid_index = iid_info.index.values
    ood_info = d.query("split_tag =='ood_val'")
    ood_index = ood_info.index.values
    test_info = d.query("split_tag =='test'")
    test_index = test_info.index.values

    iid_dataset = val_test_dataset[iid_index] if (len(iid_index)>0) else None
    ood_dataset = val_test_dataset[ood_index] if (len(ood_index)>0) else None
    test_dataset = val_test_dataset[test_index] if (len(test_index)>0) else None

IndexError: list index out of range

In [105]:
pre = "/home/jovyan/main_tankbind/dataset_prototype"
train_dataset, train_info, iid_dataset, iid_info, ood_dataset, ood_info, test_dataset, test_info = get_data_prototype(pre, "0")

['/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_train/processed/compound.pt']
['/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/data.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/protein.pt', '/home/jovyan/main_tankbind/dataset_prototype/small_val/processed/compound.pt']


In [108]:
train_dataset[0].sample_id

'ID_32'

In [109]:
info.loc[5]

sample_id                                                             ID_5
pdb_id                                                                3HB4
uniprot_id                                                          P14061
assay_id                                                                52
smiles_name                                                       smiles_2
kekule_smiles            C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...
session_au                                                       52_P14061
session_aus              52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...
pocket_com                                     [[13.393, 3.0706, -6.9976]]
pocket_name                                                         3hb4_0
value                                                               4300.0
docking_score                                                    -8.924538
mmgbsa_binding_energy                                           -54.790604
qed                      

In [91]:
info

,sample_id,pdb_id,uniprot_id,assay_id,smiles_name,kekule_smiles,session_au,session_aus,pocket_com,pocket_name,value,docking_score,mmgbsa_binding_energy,qed,split_tag,p2rank_core_pocket,session_au_usability,p_length,c_length,y_length
0,ID_0,1I5R,P14061,52,smiles_0,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,460.0,-10.036025,-69.084900,0.709065,train,True,multiton,174,24,4176
1,ID_1,3HB4,P14061,52,smiles_0,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,460.0,-8.552029,-54.705883,0.709065,train,True,multiton,164,24,3936
2,ID_2,1I5R,P14061,52,smiles_1,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,1000.0,-7.643840,-44.594658,0.766057,train,True,multiton,174,24,4176
3,ID_3,3HB4,P14061,52,smiles_1,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,1000.0,-8.719248,-55.347572,0.766057,train,True,multiton,164,24,3936
4,ID_4,1I5R,P14061,52,smiles_2,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,4300.0,-7.399574,-36.078716,0.850050,train,True,multiton,174,24,4176
5,ID_5,3HB4,P14061,52,smiles_2,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,4300.0,-8.924538,-54.790604,0.850050,train,True,multiton,164,24,3936
6,ID_6,3HB4,P14061,52,smiles_3,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,420.0,-8.748187,-69.455238,0.510305,train,True,multiton,164,24,3936
7,ID_7,1I5R,P14061,52,smiles_3,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,420.0,-7.099580,-40.027607,0.510305,train,True,multiton,174,24,4176
8,ID_8,1I5R,P14061,52,smiles_4,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.2391, 2.0646, -7.5829]]",1i5r_0,1150.0,-10.955798,-73.844841,0.587624,train,True,multiton,174,26,4524
9,ID_9,3HB4,P14061,52,smiles_4,C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H]3[C@@H]1C...,52_P14061,52_P14061_C[C@]12CC[C@@H]3C4=CC=C(O)C=C4CC[C@H...,"[[13.393, 3.0706, -6.9976]]",3hb4_0,1150.0,-9.107697,-61.683292,0.587624,train,True,multiton,164,26,4264


In [ ]:
seed = self.seed_init + epoch
info = info.groupby("session_au").sample(n=8, replace=True, random_state=seed)
info.drop_duplicates(inplace=True)
indexes = info.groupby("session_au").indices

In [74]:
iid_dataset

TankBindDataSet_prototype(4)

In [75]:
ood_dataset

In [2]:
import numpy as np

In [8]:
a = -np.exp(235235)

/tmp/ipykernel_55760/3492035550.py:1: RuntimeWarning: overflow encountered in exp
  a = -np.exp(235235)


In [9]:
a

-inf

In [11]:
import torch

In [13]:
a = torch.exp(torch.tensor(23533))

In [16]:
abs(a.item()) > 10000

True